In [1]:
!pip install pymysql
! pip install numpy
!pip install mysql-connector-python


In [2]:
import pandas as pd


In [3]:
df1=pd.read_csv(r"D:\Downloads\satellite_maintenance_enhanced.csv")

In [4]:
df1.head(3)

,cpu_load,battery_charge,radiation_exposure,temperature_interior,temperature_battery,communication_signal_strength,anomaly,timestamp,system_status,voltage,power,external_temperature
0,23.428873,86.041153,0.538071,33.689527,26.621885,-44.912752,0,2024-04-21 09:59:00,0.489860,124.967142,47.881202,21.607526
1,32.056964,88.854258,0.533670,25.218463,25.287548,-52.666438,0,2024-07-11 05:46:00,0.993943,118.617357,47.732929,23.472503
2,26.724866,75.781398,0.572964,22.324416,33.463210,-42.841038,0,2024-07-15 00:02:00,1.009366,126.476885,41.021784,22.013095


In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load Dataset
file_path = r"D:\Downloads\satellite_maintenance_enhanced.csv"  # Make sure your dataset is in the same directory
df = pd.read_csv(file_path)

# Drop timestamp since it's not useful for Random Forest
df.drop(columns=["timestamp"], inplace=True)

# Split features (X) and target (y)
X = df.drop(columns=["anomaly"])  # Features (sensor readings)
y = df["anomaly"]  # Target (0 = Normal, 1 = Anomaly)

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the Model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_rep)

# Save the Model
model_filename = "random_forest_satellite.pkl"
joblib.dump(rf_model, model_filename)

print(f"✅ Model saved as {model_filename}")


✅ Model Accuracy: 94.90%

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.96       586
           1       0.94      0.93      0.94       414

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000

✅ Model saved as random_forest_satellite.pkl


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             9 non-null      int64         
 1   cpu_load                       9 non-null      float64       
 2   battery_charge                 9 non-null      float64       
 3   radiation_exposure             9 non-null      float64       
 4   temperature_interior           9 non-null      float64       
 5   temperature_battery            9 non-null      float64       
 6   communication_signal_strength  9 non-null      float64       
 7   system_status                  9 non-null      float64       
 8   voltage                        9 non-null      float64       
 9   power                          9 non-null      float64       
 10  external_temperature           9 non-null      float64       
 11  prediction             

In [ ]:
df.columns

In [11]:
import os
import joblib

# Get the absolute path
model_path = os.path.join(os.getcwd(), "random_forest_satellite.pkl")

# Load the trained model
model = joblib.load(model_path)


In [12]:
import joblib
joblib.dump(model, "random_forest_satellite.pkl")


['random_forest_satellite.pkl']

In [13]:
import os
import joblib

# Get the absolute path of the model file
model_path = os.path.join(os.getcwd(), "random_forest_satellite.pkl")

# Load the trained model
try:
    model = joblib.load(model_path)
    print("✅ Model loaded successfully!")
except FileNotFoundError:
    print("❌ Model file not found! Check the file path and try again.")


✅ Model loaded successfully!


In [14]:
import os
print(os.path.abspath("random_forest_satellite.pkl"))


C:\Users\ASUS\random_forest_satellite.pkl


In [2]:
from fastapi import FastAPI
import joblib
import pymysql
import pandas as pd
from pydantic import BaseModel

# Load the trained model
model = joblib.load("random_forest_satellite.pkl")

# Initialize FastAPI app
app = FastAPI()

# MySQL Database Connection Config
db_config = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "@nees115",
    "database": "satellite_maintenance"
}

# Define Input Schema
class SensorData(BaseModel):
    cpu_load: float
    battery_charge: float
    radiation_exposure: float
    temperature_interior: float
    temperature_battery: float
    communication_signal_strength: float
    system_status: float
    voltage: float
    power: float
    external_temperature: float

# Root Endpoint
@app.get("/")
def home():
    return {"message": "Satellite Maintenance Prediction API is running!"}

# Prediction Endpoint
@app.post("/predict")
def predict(data: SensorData):
    try:
        # Convert input data to DataFrame
        input_df = pd.DataFrame([data.dict()])

        # Predict Anomaly
        prediction = model.predict(input_df)[0]

        # Store result in the database if it's an anomaly
        if prediction == 1:
            try:
                connection = pymysql.connect(**db_config)
                cursor = connection.cursor()

                query = """
                INSERT INTO satellite_anomalies 
                (cpu_load, battery_charge, radiation_exposure, temperature_interior, temperature_battery, 
                communication_signal_strength, system_status, voltage, power, external_temperature, anomaly)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query, (
                    data.cpu_load, data.battery_charge, data.radiation_exposure,
                    data.temperature_interior, data.temperature_battery, data.communication_signal_strength,
                    data.system_status, data.voltage, data.power, data.external_temperature, int(prediction)
                ))

                connection.commit()  # Save changes
                print("✅ Anomaly detected and stored in MySQL!")  # Debugging log

            except pymysql.MySQLError as db_err:
                print(f"❌ Database error: {db_err}")  # Print error for debugging
                return {"error": f"Database error: {str(db_err)}"}

            finally:
                cursor.close()
                connection.close()  # Close connection

        return {"anomaly": int(prediction)}
    
    except Exception as e:
        print(f"❌ Prediction failed: {e}")  # Debugging log
        return {"error": f"Prediction failed: {str(e)}"}

# Fetch Latest Sensor Data from MySQL
@app.get("/fetch-latest")
def fetch_latest_data():
    try:
        connection = pymysql.connect(**db_config)
        cursor = connection.cursor(pymysql.cursors.DictCursor)  

        query = """
        SELECT cpu_load, battery_charge, radiation_exposure, 
               temperature_interior, temperature_battery, 
               communication_signal_strength, system_status, 
               voltage, power, external_temperature 
        FROM history 
        ORDER BY timestamp DESC LIMIT 1
        """
        cursor.execute(query)
        row = cursor.fetchone()

        cursor.close()
        connection.close()  # Close connection

        if row:
            return {"latest_data": row}
        else:
            return {"message": "No data found!"}

    except pymysql.MySQLError as db_err:
        return {"error": f"Database error: {str(db_err)}"}

    except Exception as e:
        return {"error": str(e)}


In [3]:
import logging

logging.basicConfig(level=logging.INFO)

@app.post("/predict")
def predict(data: SensorData):
    try:
        input_df = pd.DataFrame([data.dict()])
        prediction = model.predict(input_df)[0]

        logging.info(f"Prediction: {prediction}")

        if prediction == 1:
            connection = pymysql.connect(**db_config)
            with connection.cursor() as cursor:
                query = """
                INSERT INTO satellite_anomalies 
                (cpu_load, battery_charge, radiation_exposure, temperature_interior, temperature_battery, 
                 communication_signal_strength, system_status, voltage, power, external_temperature, anomaly)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    data.cpu_load, data.battery_charge, data.radiation_exposure,
                    data.temperature_interior, data.temperature_battery, 
                    data.communication_signal_strength, data.system_status, 
                    data.voltage, data.power, data.external_temperature, int(prediction)
                )

                logging.info(f"Executing Query: {query} with values {values}")
                cursor.execute(query, values)
                connection.commit()
                logging.info("✅ Data inserted successfully!")
            connection.close()

        return {"anomaly": int(prediction)}

    except Exception as e:
        logging.error(f"❌ Database Insert Failed: {e}")
        return {"error": f"Prediction failed: {str(e)}"}


In [4]:
import streamlit as st
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse  # For encoding special characters in passwords

# Database Configuration
db_config = {
    "host": "localhost",  # Can also use "127.0.0.1"
    "user": "root",
    "password": "@nees115",  # Your actual password
    "database": "satellite_maintenance"
}

# Encode the password (Fixes @ symbol issue)
encoded_password = urllib.parse.quote_plus(db_config["password"])

# Function to Fetch Anomalies
def fetch_anomalies():
    try:
        # Create Database Engine
        engine = create_engine(f"mysql+pymysql://{db_config['user']}:{encoded_password}@{db_config['host']}/{db_config['database']}")
        
        # Fetch Data
        query = "SELECT * FROM satellite_anomalies ORDER BY timestamp DESC LIMIT 50"
        df = pd.read_sql(query, con=engine)

        return df if not df.empty else pd.DataFrame()
    
    except pymysql.err.OperationalError:
        st.error("🚨 Could not connect to MySQL. Make sure MySQL is running and credentials are correct.")
    
    except Exception as e:
        st.error(f"❌ Error fetching data: {e}")

    return pd.DataFrame()  # Always return an empty DataFrame

# Streamlit App
st.set_page_config(page_title="Satellite Anomalies Dashboard", layout="wide")

st.title("🚀 Satellite Anomalies Dashboard")
st.write("Monitor detected anomalies in satellite systems.")

# Refresh Data Button
if st.button("🔄 Refresh Data"):
    st.rerun()

# Fetch & Display Data
df = fetch_anomalies()

if not df.empty:
    st.write(f"### Showing {len(df)} Recent Anomalies")
    st.dataframe(df)
else:
    st.warning("⚠️ No anomalies detected yet.")


2025-03-08 15:41:22.780 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 15:41:22.782 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 15:41:25.377 
  command:

    streamlit run D:\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-08 15:41:25.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 15:41:25.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 15:41:25.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 15:41:25.383 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare m